In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import AlbertTokenizer, TFAlbertModel


In [ ]:
from google.colab import files


uploaded = files.upload()


Saving finalset.csv to finalset (3).csv


In [ ]:
df=pd.read_csv('finalset.csv')

In [ ]:
import pandas as pd
import numpy as np
import transformers
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv("finalset.csv")
category_map={
    "Knowledge": 0,
    "Comprehension": 1,
    "Application": 2,
    "Analysis": 3,
    "Synthesis": 4,
    "Evaluation": 5
}

In [ ]:
# Convert the Bloom's Taxonomy levels into one-hot encodings
# level_mapping = {'Knowledge': 0, 'Comprehension': 1, 'Application': 2, 'Analysis': 3, 'Synthesis': 4, 'Evaluation': 5}
df['category'] = df['category'].map(category_map)
# y = tf.keras.utils.to_categorical(df['category'], num_classes=6)
df.to_csv("final.csv",index=False)

In [ ]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

In [ ]:
token= tokenizer.encode_plus(
    df['text'].iloc[0],
    max_length=256,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors='pt'
)
#token.pop('token_type_ids', None) #removing token ids as they are not required in robetra

In [ ]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [ ]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['text'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [ ]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [ ]:
labels = np.zeros((len(df), 6))
labels.shape

(741, 6)

In [ ]:
labels[np.arange(len(df)), df['category'].values] = 1

In [ ]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(6,), dtype=tf.float64, name=None))>

In [ ]:
def BloomsDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [ ]:
dataset = dataset.map(BloomsDatasetMapFunction)

In [ ]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(6,), dtype=tf.float64, name=None))>

In [ ]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True)

In [ ]:
p = 0.8
train_size = int((len(df)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [ ]:
train_size


36

In [ ]:
train_dataset=dataset.take(train_size)
val_dataset=dataset.skip(train_size)
len(train_dataset),len(val_dataset)


(36, 10)

In [ ]:
Albert_model = TFAlbertModel.from_pretrained('albert-base-v2')
Albert_tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

Some layers from the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-base-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


In [ ]:
#  #Define the question classification model
# inputs = tf.keras.layers.Input(shape=(256,), dtype=tf.int32)
# attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')
# albert_output = transformers.TFAlbertModel.from_pretrained('albert-base-v2')(inputs)[0]
# pooled_output = tf.keras.layers.GlobalMaxPooling1D()(albert_output)
# outputs = tf.keras.layers.Dense(6, activation='softmax')(pooled_output)
# model = tf.keras.Model(inputs=[inputs, attn_masks], outputs=outputs)
# # model = tf.keras.Model([inputs, attn_masks])






input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

Albert_embds = Albert_model(input_ids, attention_mask=attn_masks)[1] # 0 -> last_hidden_state (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='Albert_intermediate_layer')(Albert_embds)
output_layer = tf.keras.layers.Dense(6, activation='softmax', name='albert_output_layer')(intermediate_layer) # softmax -> calcs probs of classes

Blooms_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
Blooms_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_albert_model (TFAlbertModel  TFBaseModelOutputWi  11683584   ['input_ids[0][0]',              
 )                              thPooling(last_hidd               'attention_mask[0][0]']         
                                en_state=(None, 256                                               
                                , 768),                                                       

In [ ]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-6, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
# Compile the model
Blooms_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [ ]:
hist = Blooms_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)

Epoch 1/3
36/36 [==============================] - 1943s 54s/step - loss: 1.8422 - accuracy: 0.1753 - val_loss: 1.7906 - val_accuracy: 0.2500
Epoch 2/3
36/36 [==============================] - 1945s 54s/step - loss: 1.7353 - accuracy: 0.2587 - val_loss: 1.6958 - val_accuracy: 0.2812
Epoch 3/3
36/36 [==============================] - 1887s 53s/step - loss: 1.6863 - accuracy: 0.3194 - val_loss: 1.6444 - val_accuracy: 0.3812
